In [ ]:
# This code reads the odb file and returns blade tip deflection, Failure index and 
# element number corresponding to maximum failure index.
from odbAccess import openOdb
import numpy as np
for jfjf in range(10):
    res=[]
    max_dis=[]
    Failed_ele=[]
    for i,x in enumerate(list(range(1))):
        odb = openOdb('FINAL'+str(i+1)+'.odb')
        assembly = odb.rootAssembly
        nset = assembly.nodeSets
        istep=0
        sf_13=[]
        thk=[]
        sf_23=[]
        dis=[]
        ele_Label=[]
        for step in odb.steps.values():
            istep=istep+1
            if istep==1:
               for frame in step.frames:
                   if frame.incrementNumber==1:
                       fieldr=frame.fieldOutputs['SF']
                       reac=0
                       for rvalue in fieldr.values:
                           ele_Label.append(rvalue.elementLabel)
                           sf_13.append(rvalue.data[4])
                       fields=frame.fieldOutputs['STH']
                       reac=0
                       for rvalue in fields.values:
                           thk.append(rvalue.data)
                       fieldt=frame.fieldOutputs['SF']
                       reac=0
                       for rvalue in fieldt.values:
                           sf_23.append(rvalue.data[5])
               for frame in step.frames:
                   fieldr=frame.fieldOutputs['U']
                   reac=0
                   for rvalue in fieldr.values:
                       dis.append(sqrt(((rvalue.data[0])**2)+((rvalue.data[1])**2)+((rvalue.data[2])**2)))
                   max_dis.append(max(dis))
            sf_13=np.array(sf_13)
            #print(ele_Label)
            #print(len(sf_13))
            thk=np.array(thk)
            sf_23=np.array(sf_23)
            #S_11=S_11[0:225]
            Tau_13=sf_13/thk
            Tau_23=sf_23/thk
            #print(thk)
            #print(Tau_13)
            #print(Tau_23)
            # FAILURE CRITERIA:
            F5=5*1000000
            f55=1/(F5*F5)
            RHS=((Tau_13/F5)+(Tau_23/F5))
            max_FI=max(RHS)
            RHS=list(RHS)
            #G=RHS.index(max(RHS))
            Failed_ele.append(ele_Label[RHS.index(max(RHS))])
            #print(max_FI)
            res.append(max_FI)
            odb.close()
    print(res)
    print(Failed_ele)
    print([x for x in max_dis if max_dis.index(x)%2!=0])
    break